In [8]:
%pip install peewee

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import argparse
import functools
import json
import time
import typing as t
import uuid

# pip install --upgrade psycopg2-binary
# isort: off
from playhouse.postgres_ext import *
from peewee import (
    AutoField,
    CharField,
    Model,
    MySQLDatabase,
    TextField,
    fn,
)

In [40]:
with open('/app/notebooks/paaswd.json', 'r') as config_file:
    config = json.load(config_file)
passwd = config['pgsql']['passwd']
host = config['pgsql']['host']

In [41]:
db_pg = PostgresqlExtDatabase(
    "benchmark_test",
    user="postgres_admin",
    password=passwd,
    host=host,
    port=5432,
)

In [4]:
class PGBaseModel(Model):
    class Meta:
        database = db_pg


class PGJsonDataBench(PGBaseModel):
    id = AutoField()
    jsonb_data = BinaryJSONField()
    session_id = CharField()

    class Meta:
        table_name = "jsonb_data_bench1"

In [6]:
class Timer: 
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return np.array(self.times).cumsum().tolist()

In [42]:
# 查询所有内容
query = PGJsonDataBench.select()
for i in query:
    print(i.session_id,i.jsonb_data)
    

c6eb4cd6-09b3-4b08-b20a-491aa1e870e7 {'a': 123, 'b': 456}
98493501-26fa-4607-8619-2233ba896b20 {'a': 123, 'b': 456}
54dd76fb-a097-430f-a0e4-81f8b37eb695 {'a': 123, 'b': 456}


In [23]:
# 插入json数据
data={
    "a":123,
    "b":456
}
sid = str(uuid.uuid4())
j = PGJsonDataBench(jsonb_data=data, session_id=sid)
j.save()

1

In [37]:
# 删除指定元素
q = PGJsonDataBench.select().where(PGJsonDataBench.session_id=='ccdaa6a3-403b-4c0f-8b2f-4b1f1859165f')
for item in q:
    print(item.session_id)
    item.delete_instance()

ccdaa6a3-403b-4c0f-8b2f-4b1f1859165f
